In [2]:
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf

In [3]:
base_path = './archive/real_vs_fake/real-vs-fake/'
image_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    height_shift_range=0.1,
    width_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=(0.5, 1),
    )

train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(256, 256),
    batch_size=64,
    class_mode='binary'
)


Found 100000 images belonging to 2 classes.


In [3]:
train_flow.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [4]:
train_flow[1][0].shape

(64, 256, 256, 3)

In [5]:
valid_flow = image_gen.flow_from_directory(
    base_path + 'valid/',
    target_size=(256, 256),
    batch_size=64,
    class_mode='binary'
)


Found 20000 images belonging to 2 classes.


In [6]:
test_flow = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(256, 256),
    batch_size=1,
    shuffle = False,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.


In [7]:
test_flow[1][0].shape

(1, 256, 256, 3)

In [8]:
from tensorflow.keras import models, layers
model = models.Sequential([
    
    layers.Conv2D(32, (3,3), activation='relu', input_shape = (256, 256, 3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3, 3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3, 3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(256, (3, 3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(512, (3, 3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(512, activation = 'relu'),
    layers.Dense(2, activation= 'softmax'),
    
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 128)      0

2023-03-05 23:52:21.769301: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 23:52:21.772826: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [10]:
history = model.fit(
    train_flow,
    epochs=15,
    batch_size=64,
    verbose=1,
    validation_data=valid_flow,
)

Epoch 1/15


/home/BE_Darshan/anaconda3/envs/darshan/lib/python3.10/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 179/1563 [==>...........................] - ETA: 42:22 - loss: 0.6940 - accuracy: 0.4977

In [ ]:
filename="CNN140k_test"
model.save("/home/BE_Darshan/test/models/"+filename+".h5",save_format='h5')